In [1]:
import numpy as np
import scipy as sp
import scipy.stats as stats
from scipy.stats import gaussian_kde as gkde
from scipy.special import gamma
from bokeh.plotting import figure, show, output_notebook, gridplot
from bokeh.models import Span

output_notebook()

Loading BokehJS ...

In [2]:
# Utility functions to work with the normal distribution in terms of mean and
# variance.
def normPDF(x,mu=0.0, sig2=1.0):
    return stats.norm.pdf(x,mu,np.sqrt(sig2))

def normRVS(samples=1, mu=0.0, sig2=1.0):
#     return stats.norm.rvs(mu, np.sqrt(sig2), size=samples)
    return np.random.normal(mu,np.sqrt(sig2),size=samples)

# Utility functions to work with the gamma distribution
def gammaPDF(x, alpha, beta):
    return stats.gamma.pdf(x, alpha, beta)

def gammaRVS(alpha, beta, samples=1):
#     return stats.gamma.rvs(alpha,scale=beta,size=samples)
    return np.random.gamma(alpha, beta, size=samples)

# Utility functions to work with the uniform distribution
def uniformPDF(x,lower=0.0, upper=1.0):
    return stats.uniform.pdf(x, lower, upper)

def uniformRVS(samples=1, lower=0.0, upper=1.0):
#     return stats.uniform.rvs(lower, upper, size=samples)
    return np.random.uniform(lower,upper, size=samples)

Let $X \sim 0.3\cdot\mathcal{N}(x \vert -2, 0.8) + 0.7\cdot\mathcal{N}(x \vert 2, 0.8)$

In [3]:
def p(x):
    G1 = normPDF(x, -2.0, 0.8)
    G2 = normPDF(x, 2.0, 0.8)
    return 0.3*G1 + 0.7*G2

Plot of the distribution

In [4]:
X = np.linspace(-5.0,5.0,100)
PX = [p(x) for x in X]
f = figure(title="Gaussian mixture", x_axis_label='x', y_axis_label='P(x)',
           plot_width=300, plot_height=200)
f.line(X, PX, line_width=2)
show(f)

Draw samples from the distribution

In [5]:
def draw():
    pi = uniformRVS()
    mean = -2.0 if pi <= 0.3 else 2.0
    return np.random.normal(mean, 0.8)

Parzen estimator

In [6]:
def parzen(N,h,x):
    ans = 0.0
    den = 1.0 / np.sqrt(2.0 * np.pi * (h**2))
    for n in range(N):
        xn = draw()
        num = np.exp(- ((x - xn)**2.0) / (2.0 * h**2))
        ans += num / den
    return ans / N

In [7]:
Ns = [100, 1000]
hs = [0.2, 0.3, 0.4, 0.5]
x = np.linspace(-5.0,5.0,1000)
px = [p(i) for i in x]
plots = list()

for N in Ns:
    for h in hs:
        ttl = "Parzen n: " + str(N) + ", h: " + str(h)
        f = figure(title=ttl, x_axis_label='x', y_axis_label='ph(x)')
        phx = [parzen(N,h,i) for i in x]
        f.line(x, phx, legend="Parzen estimated", color="darkgreen")
        f.line(x, px, line_width=2, color="firebrick", legend="Real dist.")
        f.legend.location = "top_left"
        plots.append(f)

grid = gridplot(plots, ncols=2, plot_width=350, plot_height=250)
show(grid)

Using a Gaussian kernel density estimator from scipy. The higher the number of samples, the better kde fits the real distribution.

In [8]:
Ns = [10, 100, 1000, 10000]
plots = list()
for n in Ns:
    samples = [draw() for i in range(n)]
    kernel = gkde(samples)
    ttl = "Gaussian kernel estimator for " + str(n) + " samples"
    f = figure(title=ttl, x_axis_label='x', y_axis_label='p(x)')
    f.line(x, kernel(x), legend="Gaussian kernel est.",
           color="darkgreen", line_width=2)
    f.line(x, px, line_width=2, color="firebrick", legend="Real dist.")
    f.legend.location = "top_left"
    plots.append(f)

grid = gridplot(plots, ncols=2, plot_width=350, plot_height=400)
show(grid)

# Sampling from Gaussian distribution

## 1. Unknown mean ($\mu$), known variance ($\sigma^2$)

In [9]:
mu = 3.0
sig2 = 0.5

Find the mean $\mu$ of $p(x) \sim \mathcal{N}(x \vert \mu, \sigma^2)$. $\mu$ (unknown) and $\sigma^2$ which is known. The **real** distribution is presented below.

In [10]:
def p(x):
    return normPDF(x, mu, sig2)

x = np.linspace(-3.0,5.0,1000)
px = [p(i) for i in x]
f = figure(title="Real distribution", x_axis_label='x', y_axis_label='p(x)',
           plot_width=300, plot_height=300)
f.line(x, px, line_width=2, color="firebrick", legend="Real dist.")
f.legend.location = "top_left"
show(f)

In [11]:
N = 30
D = normRVS(N, mu, sig2)

We assume there is a set $D= \{x_n\}^{N}_{n=1}$ of $N$ samples drawn from the distribution above.

### 1.1. Rejection sampling

The goal is to find $\mu$.

\begin{align*}
p(\mu\vert D, \sigma^2) = \frac{p(D \vert \mu, \sigma^2)p(\mu)}{p(D)}
\end{align*}

Assuming $Z_p = p(D)$ is the multiplicative constant we obtain:

\begin{align*}
p(\mu \vert D, \sigma^2) &= \frac{1}{Z_p}\widetilde{p}(\mu \vert D, \sigma^2) \\
\widetilde{p}(\mu \vert D, \sigma^2) &= p(D\vert \mu, \sigma^2)p(\mu)
\end{align*}

- Likelihood:

\begin{align*}
p(D \vert \mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} exp \left\{\frac{\sum_{n=1}^N (x_n - \mu)^2}{2\sigma^2} \right\}
\end{align*}



In [12]:
def likelihood_(mu):
    a = 1.0 / ((2.0 * np.pi * sig2) ** (N / 2.0))
    b = -np.sum((D - mu) ** 2) / (2.0 * sig2)
    return a * np.exp(b)
likelihood = np.vectorize(likelihood_)

- Prior:

\begin{align*}
q(\mu) = p(\mu) = \frac{1}{\sqrt{2\pi\sigma^2_0}} exp\left\{-\frac{(\mu - \mu_0)^2}{2\sigma^2_0}\right\}
\end{align*}

In [14]:
mu0 = 2.0 

- Posterior:

The posterior distribution using exact Bayesian inference. This will only be used to compare the accuracy.

In [16]:
def posterior(mu0, s20, s2, X, n):
    muml = np.sum(X[0 : n]) / n
    mun = (s2 * mu0 / (n * s20 + s2)) +  n * s20 * muml / (n * s20 + s2)
    sign = 1.0 / ((1.0 / s20) +  (n / s2))
    return mun, sign

muk, sigk = posterior(mu0, sig2, sig2, D, N)

def pz(z):
    return stats.norm.pdf(z, muk, np.sqrt(sigk))

Initial value for the hyper-parameter $\mu_0$. $\sigma_0^2 = \sigma^2$ beacuse the variance of the ditribution is known.

In [17]:
def q(z):
    return normPDF(z,mu0,sig2)

def draw_q(samples):
    return normRVS(samples, mu0, sig2)

- $k$:

\begin{align*}
k &= p(D \vert \mu_{ML}, \sigma^2) \\
\mu_{ML} &= \frac{1}{N}\sum_{n=1}^N x_n
\end{align*}

- Acceptance:
Let $u \sim \mathcal{U}(0,1)$, we accept a sample from $q(\mu)$ iff

\begin{align*}
u \leq \frac{p(D \vert \mu, \sigma^2)}{p(D \vert \mu_{ML}, \sigma^2)}
\end{align*}

The proportion is derived from:

\begin{align*}
\frac{\widetilde{p}(\mu)}{kq(\mu)} = \frac{p(D \vert \mu, \sigma^2)p(\mu)}{p(D \vert \mu_{ML}, \sigma^2)p(\mu)} = \frac{p(D \vert \mu, \sigma^2)}{p(D \vert \mu_{ML}, \sigma^2)}
\end{align*}
Hence, computing $k$ directly is not needed.

In [18]:
def rejectionSampling(N):
    # Draw N samples from the proposal distribution
    samples = draw_q(N)
    # Compute the maximum lakelihood for the mean
    muML = np.mean(D)
    # Compute the proportion between the probability of the data given the
    # mean and the probability of the data given muML.
    proportion = likelihood(samples) / likelihood(muML)
    u = uniformRVS(N)
    # Accept the samples for which a random sample is less than or equal to 
    # the proportion.
    return samples[u <= proportion]

### Results

We perform 4 experiments to evaluate the rejection sampling approach to find $\mu$ for the distribution. Each experiment is defined by the number of samples that rejection sampling will take to approximate $\mu$. The 4 sample sizes are $\{10, 100, 500, 1000\}$.

There is a plot for each experiment in which at least 2 samples were accepted. The dark green dotted line signals the initial value for the hyperparameter $\mu_0$. The dotted red line signals the real value of the mean for the underlying distribution. Finally, the purple dotted line is the result of the posterior using exact Bayesian inference.

In [19]:
Ns = [10, 100, 500, 1000]
x = np.linspace(0, 5, 300)

plots = list()
for n in Ns:
    samples = rejectionSampling(n)
    if len(samples) > 1:
        kernel = gkde(samples)
        ttl = "Samples=" + str(n) + " accepted=" + str(len(samples) / float(n)) 
        f = figure(title=ttl, x_axis_label='x', y_axis_label='p(x)')
        f.line(x, kernel(x), legend="Estimation",
               color="darkgreen", line_width=2)
        f.line(x, pz(x), legend='Posterior', color='purple', alpha=0.5, line_width=2, line_dash='dashed')
        realMean = Span(location=mu, dimension='height',
                        line_color='firebrick', line_dash='dashed',
                        line_width=2)
        priorMean = Span(location=mu0, dimension='height',
                        line_color='DarkOliveGreen', line_dash='dashed',
                        line_width=2)
        f.add_layout(realMean)
        f.add_layout(priorMean)
        f.legend.location = "top_left"
        plots.append(f)

grid = gridplot(plots, ncols=2, plot_width=350, plot_height=350)
show(grid)

### Cocnlusions

Rejection sampling performs well approximating $\mu$. However, when few samples are taken from the experiment of $10$ samples all of them are rejected and there is no output for that particular experiment. In the other three experiments we can see from the plots that $\mu$ is precisely approximated. Interestingly, the real value of $\mu$ is approximated with a relatively low percentage of accepted samples. 

### 1.2. Sampling importance resampling (SIR)

In [20]:
def q(z):
    return stats.norm.pdf(z, mu0, sig2) #normPDF(z, mu0, sig2)

def draw_q(n):
    return np.random.normal(mu0, sig2, size=n)# return normRVS(n, mu0, sig2)

$\widetilde{p}(\mu \vert D, \sigma^2) = p(D\vert \mu, \sigma^2)p(\mu)$

In [21]:
def pt(z):
    return likelihood(z) * q(z)

In [22]:
def computeWeights(n):
    s = draw_q(n)
    w = pt(s) / q(s)
    den = np.sum(w)
    w = w / den
    assert (abs(np.sum(w) - 1) < 1e-10)
    return s, w

In [23]:
def SIR(n, r):
    s,w = computeWeights(n)
    accu = np.copy(w)
    for i in range(1, n):
        accu[i] += accu[i - 1]

    samples = []
    for i in range(r):
        index = np.searchsorted(accu, np.random.uniform())
        samples.append(s[index])
    return samples

### Results

This time we tried SIR on three settings. Each setting is described by the pair $(n,r)$ where $n$ is the number of original samples and $r$ is the number of re-samples. The experiments considered $\{(100, 20), (1000, 100), (100000, 10000) \}$. For every experiment we present a plot of the estimated mean (in green), the posterior from Bayesian exact inference (in purple) and the initial and real means in dotted green and dotted red respectively.

In [28]:
Nexp = [100, 1000, 100000]
Nrem = [20, 100, 10000]
x = np.linspace(0, 5, 300)

plots = list()
for k in range(len(Nexp)):
    n = Nexp[k]
    r = Nrem[k]
    samples = SIR(n,r)
    
    kernel = stats.gaussian_kde(samples)
        
    ttl = "SI=" + str(n) + " R=" + str(len(samples))
    f = figure(title=ttl, x_axis_label='x', y_axis_label='p(x)')
    f.line(x, kernel(x), legend="Est. mean",
           color="darkgreen", line_width=2)
        
#     f.line(x, q(x), legend='prior', line_width=2)
    f.line(x, pz(x), legend='Posterior', color='purple', line_width=2,
           line_dash='dashed', alpha=0.5)
        
    realMean = Span(location=mu, dimension='height',
                    line_color='firebrick', line_dash='dashed',
                    line_width=2)
    priorMean = Span(location=mu0, dimension='height',
                     line_color='DarkOliveGreen', line_dash='dashed',
                     line_width=2)
    f.add_layout(realMean)
    f.add_layout(priorMean)
    f.legend.location = "top_left"
    plots.append(f)
    
grid = gridplot(plots, ncols=2, plot_width=350, plot_height=350)
show(grid)

### Conclusions

The first clear conclusion is that both SIR and exact Bayesian estimation are very close in the approximated value of $\mu$.This approximation is also close to the real value. Another conclusion in that using SIR allows to try a higher number of initial samples when compared to the rejection sampling approach.